# Objetive

Crear un dataset con todos los jugadores

Crear un dataset con cierta cantidad de partidas para analizarlas

Tener a mano los datos de todos los campeones(id, name...)

Sacar datos sobre winratios, kda, distribuciones por posición, mains...
Sacar datos sobre partidas, winratios por equipos champions, baron, dragon...

Crear un modelo de machine learning con datos sobre partidas para predecir las variables que afectan al outcome de la partida



Datos extraidos el 08/07 y 09/07 (3.291.948 total)


# Libraries

In [1]:
print('hello world')

hello world


In [2]:
import requests
import pandas as pd
import pprint
import csv
import time
import random
import itertools


In [3]:
pd.set_option('display.max_columns', None)

# Functions

## Variables Principales

In [ ]:
# Variables

server = 'euw1'
game_type = 'RANKED_SOLO_5x5'
league = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE', 'IRON', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
division = ['I', 'II', 'III', 'IV']
page = 1
api_key = ''

## Players data

In [5]:
# Función para obtener la url, dependiendo de varios parametros.

def url_players(league, division, page):
    url = f"https://{server}.api.riotgames.com/lol/league-exp/v4/entries/{game_type}/{league}/{division}?page={page}&api_key={api_key}"
    return url


In [6]:
''' Función para obtener los datos de los jugadores por ligas, divisiones y páginas con la url dada
def get_players_data_test(league, division, start_page, last_page):
    all_data = []
    for leag in league:
        for div in division:
            for page in range(start_page, last_page +1):
                url = url_players(leag, div, page)
                data = requests.get(url).json()
                all_data.extend(data)
    return all_data '''

' Función para obtener los datos de los jugadores por ligas, divisiones y páginas con la url dada\ndef get_players_data_test(league, division, start_page, last_page):\n    all_data = []\n    for leag in league:\n        for div in division:\n            for page in range(start_page, last_page +1):\n                url = url_players(leag, div, page)\n                data = requests.get(url).json()\n                all_data.extend(data)\n    return all_data '

Función para obtener los datos de los jugadores, definiendo la liga, división y la página de inicio, pero sin tener en cuenta el requests limit de la api

In [7]:
'''def get_players_data(league, division, start_page):
    all_data = []
    while True:
        url = url_players(league, division, start_page)
        data = requests.get(url).json()
        if not data:  # Si no hay datos en la página, se interrumpe el bucle
            break
        all_data.extend(data)
        start_page += 1
    return all_data'''

'def get_players_data_3(league, division, start_page):\n    all_data = []\n    while True:\n        url = url_players(league, division, start_page)\n        data = requests.get(url).json()\n        if not data:  # Si no hay datos en la página, se interrumpe el bucle\n            break\n        all_data.extend(data)\n        start_page += 1\n    return all_data'

Función agrupada para obtener por cada liga y división todas las páginas de jugadores (daba problemas...)

In [8]:
'''import time

def get_players_data_2(league, division, start_page):
    all_data = []
    page = start_page
    
    for div in division:
        while True:
            url = url_players(league, division, page)
            data = requests.get(url).json()
            
            if not data:  # Si no hay datos en la página, se interrumpe el bucle
                break
            
            all_data.extend(data)
            page += 1
            
            if page % 49 == 0:
                time.sleep(60)  # Pausa de 1 minuto cada 49 páginas
            
    return all_data '''


'import time\n\ndef get_players_data_3(league, division, start_page):\n    all_data = []\n    page = start_page\n    \n    for div in division:\n        while True:\n            url = url_players(league, division, page)\n            data = requests.get(url).json()\n            \n            if not data:  # Si no hay datos en la página, se interrumpe el bucle\n                break\n            \n            all_data.extend(data)\n            page += 1\n            \n            if page % 49 == 0:\n                time.sleep(60)  # Pausa de 1 minuto cada 49 páginas\n            \n    return all_data '

Función para obtener los datos de todas las páginas de jugadores, decidiendo liga y división y teniendo en cuenta el tiempo de espera

In [9]:
def get_players_data_3(league, division, start_page):
    all_data = [] # Creamos esta variable para que agrupe los datos de cada página
    counter = 0  # Contador para llevar el seguimiento de las solicitudes realizadas a la API
                 # con el objetivo de que no se pare la ejecución por llegar al límite
    # Comenzamos el bucle para que obtenga la información de todas las páginas por liga y división
    while True:
        url = url_players(league, division, start_page) # traemos el link de la función anterior
        data = requests.get(url).json() # obtenemos la información en formato json
        
        if not data:  # este condicional le dice al while cuando no hay datos en la página por lo tanto acaba de buscar
            break
        
        all_data.extend(data) # Mientras el bucle se ejecuta va añadiendo toda la información que se obtiene por página en la variable all_data
        counter += 1 # Aumenta el contador en 1 
        
        # Agregamos el retraso de 1 minuto cada 50 páginas (límite 100 cada 2 minutos)
        if counter % 50 == 0:
            time.sleep(60)  # Retraso de 1 minuto
        
        start_page += 1
    
    return all_data

Función para convertir la lista de diccionarios que se obtiene de la api en un csv, para poder convertirlo en dataframe sin errores

In [54]:
def save_dict_list_to_csv(data, filename):
    all_keys = set().union(*[d.keys() for d in data if isinstance(d, dict)])
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=all_keys)
        writer.writeheader()
        for d in data:
            if isinstance(d, dict):
                row = {key: d.get(key) for key in all_keys}
                writer.writerow(row)

In [11]:
# dia = get_players_data_3(league[8], division[0], 1)

In [12]:
# len(dia)

In [13]:
# save_dict_list_to_csv(dia, 'challenger')

In [14]:
# iron_4 = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\iron\iron_4')

### Concatenar los DataFrames

In [15]:
''' iron = pd.concat([iron_1, iron_2, iron_3, iron_4])
bronze = pd.concat([bronze_1, bronze_2, bronze_3, bronze_4])
silver = pd.concat([silver_1, silver_2, silver_3, silver_4])
gold = pd.concat([gold_1, gold_2, gold_3, gold_4])
plat = pd.concat([plat_1, plat_2, plat_3, plat_4])
dia = pd.concat([dia_1, dia_2, dia_3, dia_4])
'''

' iron = pd.concat([iron_1, iron_2, iron_3, iron_4])\nbronze = pd.concat([bronze_1, bronze_2, bronze_3, bronze_4])\nsilver = pd.concat([silver_1, silver_2, silver_3, silver_4])\ngold = pd.concat([gold_1, gold_2, gold_3, gold_4])\nplat = pd.concat([plat_1, plat_2, plat_3, plat_4])\ndia = pd.concat([dia_1, dia_2, dia_3, dia_4])\n'

In [16]:
'''iron = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\iron\iron.csv')
bronze = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\bronze\bronze.csv')
silver = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\silver\silver.csv')
gold = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\gold\gold.csv')
plat = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\plat\plat.csv')
dia = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\dia\dia.csv')
master = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\master\master')
grandmaster = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\grandmaster\grandmaster')
challenger = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\challenger\challenger')'''

C:\Users\Javi\AppData\Local\Temp\ipykernel_15856\1231584456.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  iron = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\iron\iron.csv')
C:\Users\Javi\AppData\Local\Temp\ipykernel_15856\1231584456.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  bronze = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\bronze\bronze.csv')
C:\Users\Javi\AppData\Local\Temp\ipykernel_15856\1231584456.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  silver = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\silver\silver.csv')
C:\Users\Javi\AppData\Local\Temp\ipykernel_15856\1231584456.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  gold = pd.read_csv(r'C:\User

In [17]:
'''players = pd.concat([iron, bronze, silver, gold, plat, dia, master, grandmaster, challenger])'''

In [18]:
'''players.to_csv('players.csv', index=False)'''

### Prueba poniendo las páginas

In [19]:

# Función para obtener los datos de los jugadores por ligas/divisiones con la url dada

def get_players_data(league, division, start_page, last_page):
    all_data = []
    for page in range(start_page, last_page +1):
        url = url_players(league, division, page)
        data = requests.get(url).json()
        all_data.extend(data)
    return all_data


In [20]:
# data2 = get_players_data(league[6], division[0], 100, 100)
# len(data2)

## Match data

### 1 - summonerName

In [21]:
iron_4 = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\iron\iron_4')

In [22]:
iron_4_names = iron_4['summonerName']

In [23]:
iron_4_names_list = list(iron_4_names)

In [24]:
len(iron_4_names_list)

19103

Sacamos porcentaje del total? o un numero fijo?

In [25]:
(len(iron_4_names_list)*1)//100

191

In [26]:
random_names_iron4 = random.sample(iron_4_names_list, 10) 

In [27]:
random_names_iron4

['nooaaaaaarrrr',
 'jejzzz',
 'DarckkuS',
 'Qianeles',
 'Fixcom00',
 'Nugytari',
 'Aminelaliara',
 'Im stµck stepbro',
 'BlooD GoSh',
 'SeaPieO6']

### 2 - Puuid

In [28]:
from urllib.parse import quote

def url_puuid(name2):
    encoded_name = quote(name2)
    url = f"https://{server}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{encoded_name}?api_key={api_key}"
    return url

In [29]:
def get_players_puuid4(lista):

    all_data = []

    for name in lista:
        url = url_puuid(name)  # Obtener la URL para el nombre actual
        data = requests.get(url).json()
        all_data.append(data)

    return all_data

In [30]:
def get_players_puuid(lista):
    all_data = []

    for name in lista:
        url = url_puuid(name)  # Obtener la URL para el nombre actual
        
        while True:
            response = requests.get(url)
            

            if response.status_code == 429:
                print("Rate Limit hit, sleeping for 10 seconds")
                time.sleep(10)
                continue  # Continuar el bucle while si se alcanza el límite de velocidad
            
            data = requests.get(url).json()
            all_data.append(data)
           
        
    return all_data


In [34]:
pd.set_option('display.max_colwidth', None)

### 3 - Match

In [36]:
def url_puuid(puuid):
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=5&api_key={api_key}"
    return url

In [37]:
def get_match_id(lista):

    all_data = []

    for puuid in lista:
        url = url_puuid(puuid)  # Obtener la URL para el nombre actual
        data = requests.get(url).json()
        all_data.append(data)

    return all_data

In [38]:
match_id_iron4 = get_match_id(iron4_puuid_df)

In [39]:
match_id_iron4 = list(itertools.chain.from_iterable(match_id_iron4))

### 4 - Match info

In [40]:
def url_matches(match_id):
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    return url

In [41]:
def get_match_data(lista3):

    all_data = []

    for match_id in lista3:
        url = url_matches(match_id)  # Obtener la URL para el nombre actual
        data = requests.get(url).json()
        all_data.append(data)

    return all_data

## Funciones

### Función entera solicitudes partidas

In [43]:
# 1 - Todo junto

league = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\players\gold\gold_2')
league_names = league['summonerName']
league_names_list = list(league_names) # creamos la lista con todos los players 

random_league_names = random.sample(league_names_list, 200) # Establecemos el número de players

# 2 - Puuid

from urllib.parse import quote

def url_puuid(name2):
    encoded_name = quote(name2)
    url = f"https://{server}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{encoded_name}?api_key={api_key}"
    return url

def get_players_puuid4(lista):

    all_data = []

    for name in lista:
        url = url_puuid(name)  # Obtener la URL para el nombre actual
        response = requests.get(url)

        if response.status_code == 429:  # Se ha excedido el límite de solicitudes
            print("1Esperando 1 minutos...")
            time.sleep(60)  # Esperar 2 minutos antes de realizar la siguiente solicitud
            response = requests.get(url)  # Realizar la solicitud nuevamente

        data = response.json()
        all_data.append(data)

    return all_data

puuid_league_names = get_players_puuid4(random_league_names) # usamos la función

puuid_league_names_df = pd.DataFrame(puuid_league_names) # lo convertimos a df

puuid_df = puuid_league_names_df['puuid'] # nos quedamos con la columna puuid para usarla más adelante


# 3 - Match

def url_puuid(puuid):
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=6&api_key={api_key}"
    return url

def get_match_id(lista):

    all_data = []

    for puuid in lista:
        url = url_puuid(puuid)  # Obtener la URL para el nombre actual
        response = requests.get(url)

        if response.status_code == 429:  # Se ha excedido el límite de solicitudes
            print("2Esperando 1 minutos...")
            time.sleep(60)  # Esperar 2 minutos antes de realizar la siguiente solicitud
            response = requests.get(url)  # Realizar la solicitud nuevamente

        data = response.json()
        all_data.append(data)

    return all_data

matches_id = get_match_id(puuid_df)

matches_id = list(itertools.chain.from_iterable(matches_id)) # convertimos la lista de listas, en una lista única


1Esperando 1 minutos...
1Esperando 1 minutos...
2Esperando 1 minutos...
2Esperando 1 minutos...


El problema se debe a que marca status a veces

In [53]:
len(matches_id)

1000

### Añadir partidas

In [312]:
for match in matches_id2:
    matches_id.append(match)

In [338]:
matches_id.append(matches_id2[-4])

### Quitar partidas

In [52]:
matches_id = matches_id[:-170]

### Quitar status

In [50]:
matches_id.remove('status')

ValueError: list.remove(x): x not in list

In [54]:
matches_id

['EUW1_6401698892',
 'EUW1_6375987677',
 'EUW1_6374825205',
 'EUW1_6374601813',
 'EUW1_6369145089',
 'EUW1_6369067697',
 'EUW1_6274809541',
 'EUW1_6274603686',
 'EUW1_6274374279',
 'EUW1_6274203019',
 'EUW1_6273519653',
 'EUW1_6273387242',
 'EUW1_6463040461',
 'EUW1_6431846032',
 'EUW1_6431305571',
 'EUW1_6416760593',
 'EUW1_6416666760',
 'EUW1_6416440198',
 'EUW1_6488943753',
 'EUW1_6450773599',
 'EUW1_6450711719',
 'EUW1_6450670101',
 'EUW1_6450617144',
 'EUW1_6422289645',
 'EUW1_6354494813',
 'EUW1_6354453805',
 'EUW1_6354416960',
 'EUW1_6354383040',
 'EUW1_6353958189',
 'EUW1_6353892824',
 'EUW1_6475042866',
 'EUW1_6474980170',
 'EUW1_6474935711',
 'EUW1_6474637271',
 'EUW1_6474596049',
 'EUW1_6474499881',
 'EUW1_6439651939',
 'EUW1_6438713648',
 'EUW1_6435955811',
 'EUW1_6435913497',
 'EUW1_6435101395',
 'EUW1_6433493329',
 'EUW1_6447473471',
 'EUW1_6421077872',
 'EUW1_6413571257',
 'EUW1_6413474132',
 'EUW1_6413425385',
 'EUW1_6413336134',
 'EUW1_6497656410',
 'EUW1_6496438938',


In [55]:

# 4 - Match info

def url_matches(match_id):
    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    return url

def get_match_data(lista):

    all_data = []
   
    for match_id in lista:
        url = url_matches(match_id)  # Obtener la URL para el nombre actual
        response = requests.get(url)

        if response.status_code == 429:  # Se ha excedido el límite de solicitudes
            print("3Esperando 2 minutos...")
            time.sleep(120)  # Esperar 2 minutos antes de realizar la siguiente solicitud
            response = requests.get(url)

        data = response.json()
        all_data.append(data)

    return all_data

data_match_challenger = get_match_data(matches_id)




3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...
3Esperando 2 minutos...


Quizas ver las partidas que faltan y volver a hacer el proceso pero solo para las partidas que faltan y luego unirlo

Probar si el rror se debe a que el dataset no tiene 100 partidas (no lo tengo claro)

### Función completa para reordenar los df

* maybe gold per minute
* BAJAR LA INFORMACIÓN SOBRE SUMMONER SPELLS Y CHAMPIONS

In [21]:
def clean_matches(lista):
    prueba = pd.DataFrame()
    columnas = ['gameId','gameDuration','seasonId','winner','firstBlood','firstTower','firstInhibitor','firstBaron','firstDragon','firstRiftHerald',
                
        't1_champ1id','t1_champ1_sum1','t1_champ1_sum2','t1_champ2id','t1_champ2_sum1','t1_champ2_sum2','t1_champ3id','t1_champ3_sum1','t1_champ3_sum2',
        't1_champ4id','t1_champ4_sum1','t1_champ4_sum2','t1_champ5id','t1_champ5_sum1','t1_champ5_sum2',
        
        't1_towerKills','t1_inhibitorKills','t1_baronKills','t1_dragonKills', 't1_riftHeraldKills',
        't1_ban1','t1_ban2','t1_ban3','t1_ban4','t1_ban5',
        
        't2_champ1id','t2_champ1_sum1','t2_champ1_sum2','t2_champ2id','t2_champ2_sum1','t2_champ2_sum2','t2_champ3id','t2_champ3_sum1','t2_champ3_sum2',
        't2_champ4id','t2_champ4_sum1','t2_champ4_sum2','t2_champ5id','t2_champ5_sum1','t2_champ5_sum2',
        
        't2_towerKills','t2_inhibitorKills','t2_baronKills','t2_dragonKills','t2_riftHeraldKills',
        't2_ban1','t2_ban2','t2_ban3','t2_ban4','t2_ban5', '','']
    loc = 0
    col = 0

#--------------------------------------------------------------#

    # MATCH DATA

    # MATCHID
    all_data = []
    for match in lista:
        datos = match['metadata']['matchId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1
    # GAMEDURATION
    all_data = []
    for match in lista:
        datos = match['info']['gameDuration']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1
    # GAMEVERSION
    all_data = []
    for match in lista:
        datos = match['info']['gameVersion']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1
    # WINNER (TRUE=1, FALSE=2)
    all_data = []
    for match in lista:
        datos = match['info']['participants'][0]['win']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)


#--------------------------------------------------------------#

    # FIRST TEAM


    loc+=1
    col+=1
    # FIRST BLOOD
    all_data = []
    for match in lista:
        jugadores = match['info']['participants']
        equipo_primera_sangre = None
        for i, jugador in enumerate(jugadores):
            if jugador['firstBloodKill'] == True:
                if i < 5:
                    equipo_primera_sangre = 1
                else:
                    equipo_primera_sangre = 2
                break 
        all_data.append(equipo_primera_sangre)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1
    # FIRST TOWER (TRUE=1, FALSE=2)
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['tower']['first']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # FIRSTINHIBITOR (TRUE=1, FALSE=2)
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['inhibitor']['first']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # firstBaron (TRUE=1, FALSE=2)
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['baron']['first']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # firstDragon (TRUE=1, FALSE=2)
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['dragon']['first']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # firstRiftHerald (TRUE=1, FALSE=2)
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['riftHerald']['first']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)


#--------------------------------------------------------------#

    # PLAYERS INFO 
        # T1
            # 1

    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][0]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][0]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][0]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 2

    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][1]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][1]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][1]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 3

    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][2]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][2]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][2]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 4
    
    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][3]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][3]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][3]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 5
    
    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][4]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][4]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][4]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)


    #--------------------------------------------------------------#

        # T1 INFO

    loc+=1
    col+=1 
    # TOWERS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['tower']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # INHIBITORS 
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['inhibitor']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # BARONS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['baron']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # DRAGONS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['dragon']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # HERALDS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['objectives']['riftHerald']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

  
        #  T1 BANS
            # 1

    loc+=1
    col+=1 
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['bans'][0]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)
    
    loc+=1
    col+=1 
            # 2
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['bans'][1]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
            # 3
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['bans'][2]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
            # 4
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['bans'][3]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
            # 5
    all_data = []
    for match in lista:
        datos = match['info']['teams'][0]['bans'][4]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)


    #--------------------------------------------------------------#

        # T2
            # 1

    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][5]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][5]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][5]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 2

    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][6]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][6]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][6]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 3

    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][7]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][7]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][7]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 4
    
    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][8]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][8]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][8]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

            # 5
    
    loc+=1
    col+=1 
    # championId 
    all_data = []
    for match in lista:
        datos = match['info']['participants'][9]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner1Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][9]['summoner1Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # summoner2Id
    all_data = []
    for match in lista:
        datos = match['info']['participants'][9]['summoner2Id']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

#--------------------------------------------------------------#

        # T2 INFO

    loc+=1
    col+=1 
    # TOWERS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['objectives']['tower']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # INHIBITORS 
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['objectives']['inhibitor']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # BARONS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['objectives']['baron']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # DRAGONS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['objectives']['dragon']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
    # HERALDS
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['objectives']['riftHerald']['kills']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

        #  T2 BANS
            # 1

    loc+=1
    col+=1 
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['bans'][0]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)
    
    loc+=1
    col+=1 
            # 2
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['bans'][1]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
            # 3
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['bans'][2]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
            # 4
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['bans'][3]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)

    loc+=1
    col+=1 
            # 5
    all_data = []
    for match in lista:
        datos = match['info']['teams'][1]['bans'][4]['championId']
        all_data.append(datos)
    all_data = pd.Series(all_data)
    prueba.insert(loc=loc, column=columnas[col], value=all_data)


    return prueba

    

## Pruebas

Cogemos los datos que nos interesan

In [56]:
gold2_clean_match_data = clean_matches(data_match_challenger)

In [58]:
gold2_clean_match_data.to_csv('gold2_clean_match_data')

### Dataset junto

In [107]:
iron1_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\iron\iron1_clean_match_data', index_col=0)
iron2_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\iron\iron2_clean_match_data', index_col=0)
iron3_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\iron\iron3_clean_match_data', index_col=0)
iron4_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\iron\iron4_clean_match_data', index_col=0)
bronze1_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\bronze\bronze1_clean_match_data', index_col=0)
bronze2_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\bronze\bronze2_clean_match_data', index_col=0)
bronze3_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\bronze\bronze3_clean_match_data', index_col=0)
bronze4_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\bronze\bronze4_clean_match_data',index_col=0)
silver1_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\silver\silver1_clean_match_data', index_col=0)
silver2_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\silver\silver2_clean_match_data', index_col=0)
silver3_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\silver\silver3_clean_match_data', index_col=0)
silver4_match = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\silver\silver4_clean_match_data', index_col=0)
gold1_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\gold\gold1_clean_match_data', index_col=0)
gold2_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\gold\gold2_clean_match_data', index_col=0)
gold3_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\gold\gold3_clean_match_data', index_col=0)
gold4_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\gold\gold4_clean_match_data', index_col=0)
plat1_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\plat\plat1_clean_match_data', index_col=0)
plat2_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\plat\plat2_clean_match_data', index_col=0)
plat3_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\plat\plat3_clean_match_data', index_col=0)
plat4_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\plat\plat4_clean_match_data', index_col=0)
dia1_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\dia\dia1_clean_match_data', index_col=0)
dia2_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\dia\dia2_clean_match_data', index_col=0)
dia3_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\dia\dia3_clean_match_data', index_col=0)
dia4_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\dia\dia4_clean_match_data', index_col=0)
master_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\master\master_clean_match_data', index_col=0)
grandmaster_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\grandmaster\grandmaster_clean_match_data', index_col=0)
challenger_matches = pd.read_csv(r'C:\Users\Javi\Desktop\Bootcamp\Proyecto_Final\data\matches\challenger\challenger_clean_match_data', index_col=0)

In [117]:
iron_matches = pd.concat([iron1_match, iron2_match, iron3_match, iron4_match])
bronze_matches = pd.concat([bronze1_match, bronze2_match, bronze3_match, bronze4_match])
silver_matches = pd.concat([silver1_match, silver2_match, silver3_match, silver4_match])
gold_matches = pd.concat([gold1_matches, gold2_matches, gold3_matches, gold4_matches])
plat_matches = pd.concat([plat1_matches, plat2_matches, plat3_matches, plat4_matches])
dia_matches = pd.concat([dia1_matches, dia2_matches, dia3_matches, dia4_matches])

In [120]:
matches_concat = pd.concat([iron_matches, bronze_matches, silver_matches, gold_matches, plat_matches, dia_matches, master_matches, grandmaster_matches, challenger_matches])

In [122]:
matches_concat.to_csv('matches.csv', index=False)

In [118]:
iron_matches.to_csv('iron_matches.csv', index=False)
bronze_matches.to_csv('bronze_matches.csv', index=False)
silver_matches.to_csv('silver_matches.csv', index=False)
gold_matches.to_csv('gold_matches.csv', index=False)
plat_matches.to_csv('plat_matches.csv', index=False)
dia_matches.to_csv('dia_matches.csv', index=False)
master_matches.to_csv('master_matches.csv', index=False)
grandmaster_matches.to_csv('grandmaster_matches.csv', index=False)
challenger_matches.to_csv('challenger_matches.csv', index=False)

### Encode

In [206]:
# iron3_data_match['firstTower'] = iron3_data_match['firstTower'].map({True : 1, False : 2})

In [105]:
# matchdata.to_csv('iron_3_matchdata')

In [91]:
# Ejemplo

champs_team1 = [(data_match_iron3[0]['info']['participants'][i]['championName'], data_match_iron3[0]['info']['participants'][i]['lane']) for i in range(0,5)]
champs_team2 =[(data_match_iron3[0]['info']['participants'][i]['championName'], data_match_iron3[0]['info']['participants'][i]['lane']) for i in range(5,10)]


# References

In [ ]:
https://www.ggrecon.com/guides/league-of-legends-rank-distribution/
https://leagueoflegends.fandom.com/wiki/Servers
https://www.leagueofgraphs.com/rankings/rank-distribution/euw

## Photos

https://zewar.files.wordpress.com/2011/06/eu2-map.jpg
https://brand.riotgames.com/es-es/league-of-legends/logos/

## Champions

In [156]:
champions_data = pd.read_json('http://ddragon.leagueoflegends.com/cdn/13.13.1/data/en_US/champion.json')

In [166]:
champions_data

,type,format,version,data
Aatrox,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Aatrox', 'key': ..."
Ahri,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Ahri', 'key': '1..."
Akali,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Akali', 'key': '..."
Akshan,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Akshan', 'key': ..."
Alistar,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Alistar', 'key':..."
...,...,...,...,...
Zeri,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Zeri', 'key': '2..."
Ziggs,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Ziggs', 'key': '..."
Zilean,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Zilean', 'key': ..."
Zoe,champion,standAloneComplex,13.13.1,"{'version': '13.13.1', 'id': 'Zoe', 'key': '14..."


In [224]:
columns = ['Name']
champions = pd.DataFrame(columns=columns)

In [225]:
champions['Name'] = champions_data.index

In [193]:
champions_data_dict = champions_data['data']

In [227]:
def clean_champions(df):
    all_keys = []
    for champion in df:
        champion_key = champion['key']
        all_keys.append(champion_key)
    all_keys = pd.Series(all_keys)
    champions.insert(loc=1, column='Id', value=all_keys)

    return all_keys

In [228]:
all_keys = clean_champions(champions_data_dict)

In [232]:
champions

,Name,Id
0,Aatrox,266
1,Ahri,103
2,Akali,84
3,Akshan,166
4,Alistar,12
...,...,...
158,Zeri,221
159,Ziggs,115
160,Zilean,26
161,Zoe,142


In [ ]:
champions.to_csv('champions')

## Table Montly players

In [11]:
montly_players = pd.DataFrame()
montly_players['year'] = ['2011', '2012', '2014', '2017', '2018',  '2019', '2020', '2021', '2022']
montly_players['players'] = [11.5, 32, 65, 100, 75, 117, 137, 149, 152]